In [1]:
import numpy as np
from IPython.display import clear_output

board_size = 15
board = np.zeros((board_size, board_size), dtype=int)
a='0'

def check_win(board, player):
    # Check horizontal win
    for i in range(board_size):
        for j in range(board_size - 4):
            if all(board[i, j + k] == player for k in range(5)):
                return True

    # Check vertical win
    for i in range(board_size - 4):
        for j in range(board_size):
            if all(board[i + k, j] == player for k in range(5)):
                return True

    # Check diagonal win (top-left to bottom-right)
    for i in range(board_size - 4):
        for j in range(board_size - 4):
            if all(board[i + k, j + k] == player for k in range(5)):
                return True

    # Check diagonal win (top-right to bottom-left)
    for i in range(board_size - 4):
        for j in range(4, board_size):
            if all(board[i + k, j - k] == player for k in range(5)):
                return True

    return False
    

def display_board(board):
    clear_output()
    print("    ", end="")
    for i in range(board_size):
        print(f"{i:2}", end=" ")
    print("\n")
    for i, row in enumerate(board):
        print(f"{i:2}  ", end="")
        print(" ".join(map(str, row)).replace('0', '_'))
    print()

def ai_move(board):
    best_score = -float('inf')
    best_move = None

    for i in range(board_size):
        for j in range(board_size):
            if board[i, j] == 0:
                board[i, j] = 2
                
                if check_win(board, 2):
                    a='win'
                    return (i, j)
                
                board[i, j] = 1
                if check_win(board, 1):
                    a='block'
                    return (i, j)

                board[i, j] = 2
                score = evaluate(board, 2)
                board[i, j] = 0

                if score > best_score:
                    best_score = score
                    best_move = (i, j)

    return best_move

def evaluate(board, player):
    ai_score = 0

    for i in range(board_size):
        for j in range(board_size - 4):
            row = board[i, j:j+5]
            col = board[j:j+5, i]
            
            if np.sum(row == player) == 4 and np.sum(row == 0) == 1:
                ai_score += 500
            elif np.sum(col == player) == 4 and np.sum(col == 0) == 1:
                ai_score += 500

            if np.sum(row == player) == 3 and np.sum(row == 0) == 2:
                ai_score += 50
            elif np.sum(col == player) == 3 and np.sum(col == 0) == 2:
                ai_score += 50

            row_rev = board[i, j:j+5][::-1]
            col_rev = board[j:j+5, i][::-1]

            if np.sum(row_rev == player) == 4 and np.sum(row_rev == 0) == 1:
                 ai_score += 500
            elif np.sum(col_rev == player) == 4 and np.sum(col_rev == 0) == 1:
                 ai_score += 500

            if np.sum(row_rev == player) == 3 and np.sum(row_rev == 0) == 2:
                ai_score += 50
            elif np.sum(col_rev == player) == 3 and np.sum(col_rev == 0) == 2:
                ai_score += 50
            
    for i in range(board_size - 4):
        for j in range(board_size - 4):
            diagonal = board[i:i+5, j:j+5]
            diagonal_rev = np.fliplr(diagonal)
            
            if np.sum(np.diag(diagonal) == player) == 4 and np.sum(np.diag(diagonal) == 0) == 1:
                ai_score += 500
            elif np.sum(np.diag(diagonal) == player) == 3 and np.sum(np.diag(diagonal) == 0) == 2:
                ai_score += 50

            if np.sum(np.diag(diagonal_rev) == player) == 4 and np.sum(np.diag(diagonal_rev) == 0) == 1:
                ai_score += 500
            elif np.sum(np.diag(diagonal_rev) == player) == 3 and np.sum(np.diag(diagonal_rev) == 0) == 2:
                ai_score += 50

    middle = board_size // 2
    for i in range(board_size):
        for j in range(board_size):
            if board[i, j] == 2:
                ai_score += 5 - abs(middle - i) - abs(middle - j)
            elif board[i, j] == 1:
                ai_score -= 5 - abs(middle - i) - abs(middle - j)

    return ai_score

def set_first_player():
    while True:
        try:
            choice = input("Who plays first? Enter 'me' for user or 'ai' for AI: ").lower()
            if choice == 'me':
                return 1
            elif choice == 'ai':
                return 2
            else:
                print("Invalid choice. Please enter 'me' or 'ai'.")
        except ValueError:
            print("Invalid input. Please enter 'me' or 'ai'.")

current_player = set_first_player()
while True:
    display_board(board)
    if current_player == 1:
        try:
            row, col = map(int, input("Enter your move (row col): "+a).split())
            if 0 <= row < board_size and 0 <= col < board_size and board[row][col] == 0:
                board[row][col] = 1
                if check_win(board, 1):
                    display_board(board)
                    print("Congratulations! You win!")
                    break
                current_player = 2
            else:
                print("Invalid move. Try again.")
        except ValueError:
            print("Invalid input. Please enter row and col as integers.")
    else:
        print("AI is making a move...")
        ai_row, ai_col = ai_move(board)
        board[ai_row][ai_col] = 2
        if check_win(board, 2):
            display_board(board)
            print("AI wins!")
            break
        current_player = 1

     0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 

 0  1 1 1 1 2 _ _ _ _ _ _ _ _ _ _
 1  _ _ _ _ _ _ _ _ _ _ _ _ _ _ _
 2  _ _ _ _ _ _ _ _ _ _ _ _ _ _ _
 3  _ _ _ _ _ _ _ _ _ _ _ _ _ _ _
 4  _ _ _ _ _ _ _ _ _ _ _ _ _ _ _
 5  _ _ _ _ _ _ _ 2 _ _ _ _ _ _ _
 6  _ _ _ _ _ _ _ 2 _ _ _ _ _ _ _
 7  _ _ _ _ _ _ _ 2 _ _ _ _ _ _ _
 8  _ _ _ _ _ _ _ 2 _ _ _ _ _ _ _
 9  _ _ _ _ _ _ _ _ _ _ _ _ _ _ _
10  _ _ _ _ _ _ _ _ _ _ _ _ _ _ _
11  _ _ _ _ _ _ _ _ _ _ _ _ _ _ _
12  _ _ _ _ _ _ _ _ _ _ _ _ _ _ _
13  _ _ _ _ _ _ _ _ _ _ _ _ _ _ _
14  _ _ _ _ _ _ _ _ _ _ _ _ _ _ _



KeyboardInterrupt: Interrupted by user